<a href="https://colab.research.google.com/github/mayait/Business-Analytics-Class/blob/main/AUTOML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. uninstall all affected packages
# 1min
!pip uninstall -y Cython scipy pyparsing scikit_learn imbalanced-learn mlxtend yellowbrick

In [ ]:
# 2. install packages to be downgraded
# Restart the runtime
# 1 min
!pip install Cython==0.29.36 scipy==1.9 pyparsing==2.4


In [ ]:
# 3. install older scikit-learn disregarding its dependencies
# 7 min
!pip install scikit-learn==0.24.2 --no-build-isolation

In [ ]:
# 4. finally install auto-sklearn
!pip install auto-sklearn

In [ ]:
# 5. then, try loading the package repeatedly until trash in its dependencies are clean
import autosklearn

In [ ]:
import autosklearn.regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)

# Automl para advertising sales (Regresión)

In [ ]:
!wget https://raw.githubusercontent.com/mayait/ClaseAnalisisDatos/main/machine_learning/datasets/advertising_and_sales_clean.csv

In [ ]:
df = pd.read_csv("advertising_and_sales_clean.csv")
df.sample()

In [ ]:
df.columns

In [ ]:
X = df[['tv', 'radio', 'social_media', 'influencer']]
y = df['sales']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
automl = autosklearn.regression.AutoSklearnRegressor(
    # Tiempo total en segundos permitido para la búsqueda y optimización del modelo.
    time_left_for_this_task=350,

    # Límite de tiempo en segundos para cada llamada al modelo.
    per_run_time_limit=120,

    # Número de núcleos de CPU a utilizar
    # Un valor de -1 significa que se utilizarán todos los núcleos disponibles.
    n_jobs=-1,

    # Métrica de evaluación para optimizar el modelo.
    # En este caso, se utiliza el error cuadrático medio (Mean Squared Error).
    metric=autosklearn.metrics.mean_squared_error,
)

automl.fit(X_train, y_train)

In [ ]:
# Predicciones en el conjunto de entrenamiento
y_train_pred = automl.predict(X_train)

# Cálculo de MSE y R2 para el conjunto de entrenamiento
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

# Predicciones en el conjunto de prueba
y_test_pred = automl.predict(X_test)

# Cálculo de MSE y R2 para el conjunto de prueba
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Imprimir resultados
print("Conjunto de entrenamiento:")
print("  Mean squared error:", mse_train)
print("  R2 score:", r2_train)

print("\nConjunto de prueba:")
print("  Mean squared error:", mse_test)
print("  R2 score:", r2_test)

# Umbral para determinar si hay sobreajuste o subajuste
threshold = 0.2

# Calcular la diferencia entre R2 y MSE en los conjuntos de entrenamiento y prueba
r2_difference = abs(r2_train - r2_test)
mse_difference = abs(mse_train - mse_test)


### Imprime los resultados de los diferentes modelos:

- **ARDRegression** (Automatic Relevance Determination Regression) En esencia, ARDRegression es una versión más generalizada de Ridge Regression que permite aprender automáticamente la importancia de las características durante el entrenamiento.

- **HistGradientBoostingRegressor** es un enfoque de aprendizaje ensamblado que construye secuencialmente árboles de decisión débiles y los combina para formar un modelo final más preciso. En cada iteración, el algoritmo entrena un nuevo árbol para corregir los errores cometidos por el conjunto actual de árboles.



In [ ]:
def pretty_print_models(model_dict):
    for model_id, model_info in model_dict.items():
        print(f"Model ID: {model_id}")
        print(f"  Rank: {model_info['rank']}")
        print(f"  Cost (MSE): {model_info['cost']:.2f}")
        print(f"  Ensemble Weight: {model_info['ensemble_weight']:.2f}")
        print(f"  Data Preprocessor: {type(model_info['data_preprocessor']).__name__}")
        print(f"  Feature Preprocessor: {type(model_info['feature_preprocessor']).__name__}")
        print(f"  Regressor: {type(model_info['regressor']).__name__}")
        print(f"  Sklearn Regressor: {model_info['sklearn_regressor']}\n")

models = automl.show_models()
pretty_print_models(models)


In [ ]:
leaderboard = automl.leaderboard(sort_by="rank", ensemble_only=True)
leaderboard

# 🌶️ 🐕 Aplica Automl para Credit (Clasificación)

In [ ]:
!wget https://raw.githubusercontent.com/mayait/ClaseAnalisisDatos/main/DesitionTree/data/bank/bank-full.csv

In [ ]:
df = pd.read_csv('bank-full.csv', sep=';')
df.sample(10)